# This notebook shows the data analysis on the Kaggle dataset ["Where it Pays to Attend College"](https://www.kaggle.com/datasets/wsj/college-salaries)

In [1]:
# Install all dependencies
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install pyspark

In [2]:
# Initialize Spark
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 17:54:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
degrees = spark.read.csv("./data/degrees-that-pay-back.csv", header='true')
college_types = spark.read.csv("./data/salaries-by-college-type.csv", header='true')
region = spark.read.csv("./data/salaries-by-region.csv", header='true')

root
 |-- School Name: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Starting Median Salary: string (nullable = true)
 |-- Mid-Career Median Salary: string (nullable = true)
 |-- Mid-Career 10th Percentile Salary: string (nullable = true)
 |-- Mid-Career 25th Percentile Salary: string (nullable = true)
 |-- Mid-Career 75th Percentile Salary: string (nullable = true)
 |-- Mid-Career 90th Percentile Salary: string (nullable = true)

